In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
# load data
import zipfile
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")

# Data preprocessing

In [ ]:
data = os.listdir('./train') #list of all files

In [ ]:
# create labels
labels = []
for file_name in data:
    label, _, __ = file_name.split('.')
    labels.append(label)

In [ ]:
# create dataset
train = pd.DataFrame({'files' : data, 'label' : labels })
train.head()

In [ ]:
# shuffle data
train = train.sample(frac=1)

In [ ]:
# data augmentation
train_d = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True) 

valid_d = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True) 

In [ ]:
# split data into training and validation sets
train_generator = train_d.flow_from_dataframe(train.iloc[:20000, :],
                                              directory='./train', 
                                              x_col="files",
                                              y_col="label",
                                              target_size=(224, 224),
                                              class_mode='binary')

valid_generator = valid_d.flow_from_dataframe(train.iloc[20000:, :],
                                              directory='./train', 
                                              x_col="files",
                                              y_col="label",
                                              target_size=(224, 224), 
                                              class_mode='binary')

# Define the Network

In [ ]:

def my_model():
    inputs = tf.keras.layers.Input(shape=(224,224, 3))
    x = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    x = tf.keras.layers.Dense(1, activation="sigmoid", name="classification")(x)
    return tf.keras.Model(inputs=inputs, outputs = x)


In [ ]:
def train_one_step( x, y):
    with tf.GradientTape() as tape:
        predictions = model(x)
        loss = train_loss(y,predictions)
    grads = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_accuracy.update_state(y,predictions)
    return loss

In [ ]:
model = my_model()
optimizer=tf.keras.optimizers.Adam()
epochs=3
train_loss=tf.keras.losses.BinaryCrossentropy()
train_accuracy=tf.keras.metrics.BinaryAccuracy()
val_loss=tf.keras.losses.BinaryCrossentropy()
val_accuracy=tf.keras.metrics.BinaryAccuracy()
@tf.function
def train():
    loss = 0.0
    for epoch in range(epochs):
        for x, y in train_generator:
            y = tf.reshape(y, (32,1))
            loss = train_one_step(x, y)
    for x, y in valid_generator:
        predictions = model(x)
        loss_v = val_loss(y, predictions)
        val_accuracy.update_state(y,predictions)
    tf.print('train loss: ', loss, ' valid loss: ', loss_v)
  


In [ ]:
train()
print('train accuracy: ', train_accuracy.result().numpy(), ' valid accuracy: ', valid_accuracy.result().numpy())